In [1]:
!pip install -q  torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate datasets
# import torch
import datasets
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
# from peft import LoraConfig
# from trl import SFTTrainer
from datasets import Dataset, DatasetDict, load_dataset


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Load dataset from HuggingFace**

In [18]:
# Dataset
data_name = "jiuyuan/Policy_AI"
training_data = load_dataset(data_name, split="train")
test_data = load_dataset(data_name, split="test")

In [11]:
df_test = test_data.to_pandas()
df_test.to_csv('/content/drive/MyDrive/policy_AI/prompt_engineering/test.csv',index=False)

In [7]:
df_test[['input']].to_csv('prompts.csv')

In [27]:
import pandas as pd
df_prompt = pd.read_csv("/content/drive/MyDrive/policy_AI/prompt_engineering/test_instruction.csv" )
df_prompt['prompt_engineering'] = df_prompt['prompt_engineering'].apply(lambda x:x[1:-1])
df_prompt

,input,prompt_engineering,output
0,Why is international cooperation on AI important?,nswer this question based on your knowledge: W...,International cooperation on AI is crucial due...
1,Can you describe the approach taken by the res...,nswer this question based on your knowledge: C...,The research team prepared generic interview p...
2,What concerns do critics have regarding the EU...,nswer this question based on your knowledge: W...,"Critics, including EU businesses, have express..."
3,How does the European Union classify AI system...,nswer this question based on your knowledge: H...,The European Union's AI Act classifies AI syst...
4,What are the Universal Guidelines for Artifici...,nswer this question based on your knowledge: W...,The Universal Guidelines include rights to tra...
5,What role do whistleblowers and complaints pla...,nswer this question based on your knowledge: ...,Whistleblowers and individual complaints are c...
6,How's UK's approach different from that of the...,Answer this question based on your knowledge: ...,"Unlike the European Union (EU), the UK’s appro..."
7,How does Japan's approach to AI regulation dif...,nswer this question based on your knowledge: ...,"Japan focuses on a risk-based, agile, and mult..."
8,"What are foundational AI models, and why are t...",nswer this question based on your knowledge: ...,"Foundational AI models, such as large language..."
9,What additional measures are needed for intern...,nswer this question based on your knowledge: ...,The article suggests that more ambitious trade...


In [43]:

def prompt_engineering(df):
    output_texts = []

    for i in range(len(df['input'])):
        row = df.iloc[i]
        text = f"### Question: {row['input']}\n Use this template to answer the question: {row['prompt_engineering']} \n ### Answer: "
        output_texts.append(text)
    return output_texts

df_prompt['prompt']= prompt_engineering(df_prompt)
print(df_prompt.iloc[0]['prompt'])
df_prompt


### Question: Why is international cooperation on AI important?
 Use this template to answer the question: nswer this question based on your knowledge: Why is international cooperation on AI important 
 ### Answer: 


,input,prompt_engineering,output,prompt
0,Why is international cooperation on AI important?,nswer this question based on your knowledge: W...,International cooperation on AI is crucial due...,### Question: Why is international cooperation...
1,Can you describe the approach taken by the res...,nswer this question based on your knowledge: C...,The research team prepared generic interview p...,### Question: Can you describe the approach ta...
2,What concerns do critics have regarding the EU...,nswer this question based on your knowledge: W...,"Critics, including EU businesses, have express...",### Question: What concerns do critics have re...
3,How does the European Union classify AI system...,nswer this question based on your knowledge: H...,The European Union's AI Act classifies AI syst...,### Question: How does the European Union clas...
4,What are the Universal Guidelines for Artifici...,nswer this question based on your knowledge: W...,The Universal Guidelines include rights to tra...,### Question: What are the Universal Guideline...
5,What role do whistleblowers and complaints pla...,nswer this question based on your knowledge: ...,Whistleblowers and individual complaints are c...,### Question: What role do whistleblowers and ...
6,How's UK's approach different from that of the...,Answer this question based on your knowledge: ...,"Unlike the European Union (EU), the UK’s appro...",### Question: How's UK's approach different fr...
7,How does Japan's approach to AI regulation dif...,nswer this question based on your knowledge: ...,"Japan focuses on a risk-based, agile, and mult...",### Question: How does Japan's approach to AI ...
8,"What are foundational AI models, and why are t...",nswer this question based on your knowledge: ...,"Foundational AI models, such as large language...","### Question: What are foundational AI models,..."
9,What additional measures are needed for intern...,nswer this question based on your knowledge: ...,The article suggests that more ambitious trade...,### Question: What additional measures are nee...


In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# Model and tokenizer names
import torch
base_model_name = "NousResearch/Llama-2-7b-chat-hf"
# refined_model = "llama-2-7b-mlabonne-enhanced"

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

## **Generations & Evaluation**

In [44]:
df_test_all=test_data.to_pandas()
df_train=training_data.to_pandas()
df_test_all

,input,output
0,Why is international cooperation on AI important?,International cooperation on AI is crucial due...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...
2,What concerns do critics have regarding the EU...,"Critics, including EU businesses, have express..."
3,How does the European Union classify AI system...,The European Union's AI Act classifies AI syst...
4,What are the Universal Guidelines for Artifici...,The Universal Guidelines include rights to tra...
5,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...
6,How's UK's approach different from that of the...,"Unlike the European Union (EU), the UK’s appro..."
7,How does Japan's approach to AI regulation dif...,"Japan focuses on a risk-based, agile, and mult..."
8,"What are foundational AI models, and why are t...","Foundational AI models, such as large language..."
9,What additional measures are needed for intern...,The article suggests that more ambitious trade...


In [45]:
df_test_all['output'].apply(lambda x:len(x.split())).describe(), df_train['output'].apply(lambda x:len(x.split())).describe()

(count    19.000000
 mean     40.631579
 std      11.591235
 min      17.000000
 25%      33.000000
 50%      41.000000
 75%      48.500000
 max      61.000000
 Name: output, dtype: float64,
 count      75.000000
 mean       70.000000
 std       135.589205
 min        17.000000
 25%        30.000000
 50%        39.000000
 75%        52.000000
 max      1020.000000
 Name: output, dtype: float64)

In [46]:
from tqdm.notebook import tqdm
import gc
generations_prompt_engineer_controlcodes =[]
for i in tqdm(range(len(df_prompt)), "generating..."):
  prompt = df_prompt.iloc[i]['prompt']
  # Generate predictions
  inputs = llama_tokenizer(prompt, return_tensors='pt')
  inputs = inputs.to("cuda")
  output = base_model.generate(**inputs, max_new_tokens=200,temperature=0.2)
  response = llama_tokenizer.decode(output[0].tolist())
  print(response)
  # break
  generations_prompt_engineer_controlcodes.append(response)
  del inputs, output
  gc.collect()
  torch.cuda.empty_cache()

generating...:   0%|          | 0/19 [00:00<?, ?it/s]

<s> ### Question: Why is international cooperation on AI important?
 Use this template to answer the question: nswer this question based on your knowledge: Why is international cooperation on AI important 
 ### Answer:  International cooperation on AI is important for several reasons: 

1. **Sharing knowledge and expertise**: By working together, countries can pool their resources and expertise to advance the field of AI. This can lead to more innovative and effective AI solutions, as well as a more robust global AI ecosystem.

2. **Addressing ethical and social implications**: AI raises a range of ethical and social implications, such as privacy, bias, and job displacement. International cooperation can help countries address these issues collectively, developing common standards and guidelines that promote responsible AI development and use.

3. **Fostering trust and security**: As AI becomes more pervasive in society, it's essential to ensure that AI systems are secure and trustwort

In [47]:
df_test['prompt_engineering_control_codes_generations']=generations_prompt_engineer_controlcodes
df_test

,input,output,prompt_engineering_templating_generations,prompt_engineering_control_codes_generations
0,Why is international cooperation on AI important?,International cooperation on AI is crucial due...,International cooperation on AI iimportant f...,<s> ### Question: Why is international coopera...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...,The research team took a multi-faceted appro...,<s> ### Question: Can you describe the approac...
2,What concerns do critics have regarding the EU...,"Critics, including EU businesses, have express...",Criticof the EU'AI Act have raised several c...,<s> ### Question: What concerns do critics hav...
3,How does the European Union classify AI system...,The European Union's AI Act classifies AI syst...,The European Union (EU) haclassified AI syst...,<s> ### Question: How does the European Union ...
4,What are the Universal Guidelines for Artifici...,The Universal Guidelines include rights to tra...,The Universal Guidelinefor Artificial Intell...,<s> ### Question: What are the Universal Guide...
5,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...,Whistleblowerand complaintplay a crucial rol...,<s> ### Question: What role do whistleblowers ...
6,How's UK's approach different from that of the...,"Unlike the European Union (EU), the UK’s appro...",The UK'approach to data protection idifferen...,<s> ### Question: How's UK's approach differen...
7,How does Japan's approach to AI regulation dif...,"Japan focuses on a risk-based, agile, and mult...",Japan'approach to AI regulation differfrom t...,<s> ### Question: How does Japan's approach to...
8,"What are foundational AI models, and why are t...","Foundational AI models, such as large language...",Foundational AI modelare the fundamental bui...,<s> ### Question: What are foundational AI mod...
9,What additional measures are needed for intern...,The article suggests that more ambitious trade...,International cooperation on foundational AI...,<s> ### Question: What additional measures are...


In [48]:
# Model
raw_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
raw_model.config.use_cache = False
raw_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [49]:
from tqdm.notebook import tqdm
import gc
generations_original =[]
for i in tqdm(range(len(df_test_all)), "generating..."):
  prompt = f"### Question: {df_test_all['input'][i]}\n ### Answer: "
  # Generate predictions
  inputs = llama_tokenizer(prompt, return_tensors='pt')
  inputs = inputs.to("cuda")
  output = raw_model.generate(**inputs, max_new_tokens=100,temperature=0.2)
  response = llama_tokenizer.decode(output[0].tolist())
  # print(response)
  # break
  generations_original.append(response)
  del inputs, output
  gc.collect()
  torch.cuda.empty_cache()

generations_original

generating...:   0%|          | 0/19 [00:00<?, ?it/s]

['<s> ### Question: Why is international cooperation on AI important?\n ### Answer:  International cooperation on AI is important for several reasons:\n\n1. **Sharing knowledge and expertise**: By working together, countries can pool their resources and expertise to advance the field of AI more quickly and effectively. This can lead to the development of new technologies and applications that benefit society as a whole.\n2. **Addressing ethical and social implications**: AI raises a range of ethical and social implications, such as privacy',
 '<s> ### Question: Can you describe the approach taken by the research team in preparing for the interviews with academia and industry experts in the AI assessment?\n ### Answer:  The research team took a multi-faceted approach in preparing for the interviews with academia and industry experts in the AI assessment. Here are some of the key steps we took:\n\n1. Literature Review: We conducted a comprehensive literature review to identify the key th

In [53]:
df_test_all['original_llama_generations']=generations_original
df_test_all

,input,output,prompt_engineering_templating_generations,prompt_engineering_control_codes_generations,original_llama_generations
0,Why is international cooperation on AI important?,International cooperation on AI is crucial due...,International cooperation on AI iimportant f...,<s> ### Question: Why is international coopera...,<s> ### Question: Why is international coopera...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...,The research team took a multi-faceted appro...,<s> ### Question: Can you describe the approac...,<s> ### Question: Can you describe the approac...
2,What concerns do critics have regarding the EU...,"Critics, including EU businesses, have express...",Criticof the EU'AI Act have raised several c...,<s> ### Question: What concerns do critics hav...,<s> ### Question: What concerns do critics hav...
3,How does the European Union classify AI system...,The European Union's AI Act classifies AI syst...,The European Union (EU) haclassified AI syst...,<s> ### Question: How does the European Union ...,<s> ### Question: How does the European Union ...
4,What are the Universal Guidelines for Artifici...,The Universal Guidelines include rights to tra...,The Universal Guidelinefor Artificial Intell...,<s> ### Question: What are the Universal Guide...,<s> ### Question: What are the Universal Guide...
5,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...,Whistleblowerand complaintplay a crucial rol...,<s> ### Question: What role do whistleblowers ...,<s> ### Question: What role do whistleblowers ...
6,How's UK's approach different from that of the...,"Unlike the European Union (EU), the UK’s appro...",The UK'approach to data protection idifferen...,<s> ### Question: How's UK's approach differen...,<s> ### Question: How's UK's approach differen...
7,How does Japan's approach to AI regulation dif...,"Japan focuses on a risk-based, agile, and mult...",Japan'approach to AI regulation differfrom t...,<s> ### Question: How does Japan's approach to...,<s> ### Question: How does Japan's approach to...
8,"What are foundational AI models, and why are t...","Foundational AI models, such as large language...",Foundational AI modelare the fundamental bui...,<s> ### Question: What are foundational AI mod...,<s> ### Question: What are foundational AI mod...
9,What additional measures are needed for intern...,The article suggests that more ambitious trade...,International cooperation on foundational AI...,<s> ### Question: What additional measures are...,<s> ### Question: What additional measures are...


In [52]:
df_test_all = df_test
df_test_all['original_llama_generations']= df_test_all['original_llama_generations'].apply(lambda x: x.split('Answer:', 1)[-1] if 'Answer:' in x else x)
df_test_all['original_llama_generations']= df_test_all['original_llama_generations'].apply(lambda x: x.replace('</s>', '').replace('<s>', '').replace('</s>s', '').replace('<s/>', '').replace('s ', ''))
df_test_all['prompt_engineering_templating_generations']= df_test_all['prompt_engineering_templating_generations'].apply(lambda x: x.split('Answer:', 1)[-1] if 'Answer:' in x else x)
df_test_all['prompt_engineering_templating_generations']= df_test_all['prompt_engineering_templating_generations'].apply(lambda x: x.replace('</s>', '').replace('<s>', '').replace('</s>s', '').replace('<s/>', '').replace('s ', '').replace('\n', ''))
df_test_all

KeyError: 'original_llama_generations'

In [ ]:
df_test_all.to_csv('/content/drive/MyDrive/policy_AI/prompt_engineering/prompt_engineering_templating_results.csv',index=False)

## **Metrics**

In [54]:
df_test_all = pd.read_csv('/content/drive/MyDrive/policy_AI/prompt_engineering/prompt_engineering_templating_results.csv')

In [38]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install nltk rouge -q
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge


In [39]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
def calculate_bleu_rouge(reference, candidate):
    """
    Calculate BLEU and ROUGE scores for a pair of strings.

    :param reference: The reference string (ground truth).
    :param candidate: The candidate string (generated text).
    :return: A dictionary containing BLEU and ROUGE scores.
    """
    # Preprocessing the strings: tokenizing into words
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    # Calculating BLEU score
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens)

    # Calculating ROUGE score
    rouge = Rouge()
    rouge_score = rouge.get_scores(candidate, reference)[0]

    return np.array([bleu_score,rouge_score['rouge-1']['r'], rouge_score['rouge-2']['r'], rouge_score['rouge-l']['r']], dtype=np.float32)

# Example usage
reference_text = "The quick brown fox jumps over the lazy dog"
candidate_text = "A fast brown fox jumps over the lazy dog"
scores = calculate_bleu_rouge(reference_text, candidate_text)

print(scores)


[0.7259795 0.7777778 0.75      0.7777778]


In [55]:
raw_model_metrics = df_test_all.apply(lambda x: calculate_bleu_rouge(x['output'], x['prompt_engineering_templating_generations']), axis=1)
raw_model_metrics = np.array(raw_model_metrics).mean(-1)
raw_model_metrics

array([0.00577388, 0.2923511 , 0.05147372, 0.25596383], dtype=float32)

In [57]:
df_metrics = pd.DataFrame([raw_model_metrics], columns=['BLEU_Score', 'ROUGE-1', 'ROUGE-2', 'ROUGE-l'])
df_metrics.index  = ['Raw_llama2']
df_metrics

,BLEU_Score,ROUGE-1,ROUGE-2,ROUGE-l
Raw_llama2,0.005774,0.292351,0.051474,0.255964


In [58]:
prompt_engineering_templating_metrics = df_test_all.apply(lambda x: calculate_bleu_rouge(x['output'], x['prompt_engineering_templating_generations']), axis=1)
prompt_engineering_templating_metrics = np.array(prompt_engineering_templating_metrics).mean(-1)
prompt_engineering_templating_metrics

array([0.00577388, 0.2923511 , 0.05147372, 0.25596383], dtype=float32)

In [59]:
df_metrics = pd.DataFrame([prompt_engineering_templating_metrics], columns=['BLEU_Score', 'ROUGE-1', 'ROUGE-2', 'ROUGE-l'])
df_metrics.index  = ['prompt_engineering_templating_metrics']
df_metrics

,BLEU_Score,ROUGE-1,ROUGE-2,ROUGE-l
prompt_engineering_templating_metrics,0.005774,0.292351,0.051474,0.255964
